In [1]:
import beanmachine.ppl as bm
import beanmachine.ppl.experimental.inference_compilation.ic_infer as ic_infer
from beanmachine.ppl.model.statistical_model import StatisticalModel
from beanmachine.ppl.model.utils import Mode

import torch
import torch.distributions as dist
import torch.optim as optim

In [3]:
class NuisanceParameterModel:
    @bm.random_variable
    def x(self):
        return dist.Normal(0, 1)
    
    @bm.random_variable
    def nuisance(self, i):
        return dist.Normal(0, 1)
    
    @bm.random_variable
    def y(self):
        return dist.Normal(0, 1)
    
    @bm.random_variable
    def noisy_sq_length(self):
        return dist.Normal(self.x()**2 + self.y()**2, 0.01)
    

model = NuisanceParameterModel()
ic = ic_infer.ICInference().compile(
    [model.noisy_sq_length()],
    num_worlds=1e3,
    optimizer_func=lambda x: optim.Adam(x, lr=1e-3),
)
samples = ic.infer(
    [model.x()],
    {model.noisy_sq_length(): torch.tensor(1.0)},
    num_samples=100,
    num_chains=1,
)

Loss: tensor([46.1922], grad_fn=<AddBackward0>)


In [4]:
bm.Diagnostics(samples).summary()

,avg,std,2.5%,50%,97.5%,n_eff
"x(<__main__.NuisanceParameterModel object at 0x7f739623c3d0>,)[]",0.066708,0.042006,0.019782,0.076276,0.126133,5.504583


In [5]:
num_params = 0
for pg in ic._optimizer.param_groups:
    for p in pg['params']:
        num_params += p.numel()
num_params

277

In [7]:
import pyprob
import pyprob.distributions

class NuisanceModel(pyprob.Model):
    def __init__(self):
        super().__init__(name='Nuisance parameters')

    def forward(self):
        x = pyprob.sample(pyprob.distributions.Normal(0, 1))
        nuisance = [pyprob.sample(pyprob.distributions.Normal(0, 1)) for _ in range(100)]
        y = pyprob.sample(pyprob.distributions.Normal(0, 1))

        noisy_sq_length_lik = pyprob.distributions.Normal(x**2 + y**2, 0.01)

        pyprob.observe(noisy_sq_length_lik, name='noisy_sq_length_lik')
        return x
    
model = NuisanceModel()
model.learn_inference_network(num_traces=int(1e3),
                              observe_embeddings={'noisy_sq_length_lik' : {'dim' : 4}},
                              inference_network=pyprob.InferenceNetwork.LSTM)

Creating new inference network...
Observable noisy_sq_length_lik: reshape not specified, using shape torch.Size([]).
Observable noisy_sq_length_lik: using embedding dim torch.Size([4]).
Observable noisy_sq_length_lik: observe embedding not specified, using the default FEEDFORWARD.
Observable noisy_sq_length_lik: embedding depth not specified, using the default 2.
Observe embedding dimension: 4
Train. time | Epoch| Trace     | Init. loss| Min. loss | Curr. loss| T.since min | Learn.rate| Traces/sec
New layers, address: 16__forward__x__Normal__1, distribution: Normal
New layers, address: 24__forward__<listcomp>__?__Normal__1, distribution: Normal
New layers, address: 24__forward__<listcomp>__?__Normal__2, distribution: Normal
New layers, address: 24__forward__<listcomp>__?__Normal__3, distribution: Normal
New layers, address: 24__forward__<listcomp>__?__Normal__4, distribution: Normal
New layers, address: 24__forward__<listcomp>__?__Normal__5, distribution: Normal
New layers, address: 24

In [8]:
posterior = model.posterior_results(
    num_traces=100, # the number of samples estimating the posterior
    inference_engine=pyprob.InferenceEngine.IMPORTANCE_SAMPLING_WITH_INFERENCE_NETWORK, # specify which inference engine to use
    observe={'noisy_sq_length_lik': 1.0}
)

Time spent  | Time remain.| Progress             | Trace   | Traces/sec
0d:00:00:27 | 0d:00:00:00 | #################### | 100/100 | 3.64       


In [9]:
posterior._effective_sample_size

tensor(1.0485, dtype=torch.float64)